<a href="https://colab.research.google.com/github/mukerem/AmharicHandwrittenDigitRecognitionCNN/blob/main/Handwritten_Amharic_Digit_Recognition_Using_Deep_Learning-18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Load library**

In [ ]:
!unzip '/content/gdrive/MyDrive/PreProcessV1.zip' -d "/content/gdrive/MyDrive/"

In [23]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)


warnings.filterwarnings('ignore')

In [24]:
%cd /content/gdrive/MyDrive/PreProcessV1/

/content/gdrive/MyDrive/PreProcessV1


In [25]:
batch_size = 32
img_height = 32
img_width = 32

In [26]:
data_dir = "/content/gdrive/MyDrive/PreProcessV1/"

### **keras**


In [27]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  color_mode="grayscale",
  batch_size=batch_size)

Found 51952 files belonging to 20 classes.
Using 41562 files for training.


In [28]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  color_mode="grayscale",
  batch_size=batch_size)

Found 51952 files belonging to 20 classes.
Using 10390 files for validation.


In [29]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [51]:
model = keras.Sequential([
            keras.Input(shape = (32, 32, 1)),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv1"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv2"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv3"),
            layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv4"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv5"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv6"),
            layers.MaxPooling2D(name="Maxpool2"),
            layers.Dropout(0.25, input_shape=(2,), name="Dropout1"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv7"),
            layers.MaxPooling2D(name="Maxpool3"),
            layers.Dropout(0.3, input_shape=(2,), name="Dropout2"),
            layers.Conv2D(128, 3, padding = 'same', activation = 'relu', name="Conv8"),
            layers.Flatten(name="Flatten1"),
            layers.Dense(256, activation = 'relu', name="Dense1"),
            layers.Dense(64, activation = 'relu', name="Dense2"),
            layers.Dense(20, activation = 'softmax', name="Dense3"),
])

In [52]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 32, 32, 32)        320       
                                                                 
 Conv2 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Conv3 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Maxpool1 (MaxPooling2D)     (None, 16, 16, 32)        0         
                                                                 
 Conv4 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 Conv5 (Conv2D)              (None, 16, 16, 64)        36928     
                                                                 
 Conv6 (Conv2D)              (None, 16, 16, 64)       

In [53]:
# nadam = tf.keras.optimizers.Nadam(
#     learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
# )

In [54]:
# adamax = tf.keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [55]:
# sgd = tf.keras.optimizers.SGD(
#     learning_rate=0.01,
#     momentum=0.0,
#     nesterov=False,
#     name='SGD',
# )

In [56]:
adam =  keras.optimizers.Adam(lr = 1e-3)

In [57]:
model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        optimizer = adam,
        metrics = ['acc'],
)

In [58]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.5, min_lr = 1e-7)

In [59]:
checkpoint = ModelCheckpoint("best_model_epoch_{epoch:02d}-{100*val_acc:.2f}.h5", monitor='val_acc', verbose=1,
    save_best_only=True, mode='auto', save_freq=1)

In [60]:
classifier = model.fit(train_ds, batch_size = batch_size, 
                                 epochs = 30,
                                 callbacks=[reduce_lr, checkpoint],
                                 validation_data = val_ds)

Epoch 1/30
1299/1299 [==============================] - ETA: 0s - loss: 1.3663 - acc: 0.5802
Epoch 1: val_acc improved from -inf to 0.84793, saving model to best_model_epoch_01-0.58.h5
1299/1299 [==============================] - 28s 21ms/step - loss: 1.3663 - acc: 0.5802 - val_loss: 0.5772 - val_acc: 0.8479 - lr: 0.0010
Epoch 2/30
1297/1299 [============================>.] - ETA: 0s - loss: 0.4580 - acc: 0.8772
Epoch 2: val_acc improved from 0.84793 to 0.91049, saving model to best_model_epoch_02-0.35.h5
1299/1299 [==============================] - 28s 21ms/step - loss: 0.4578 - acc: 0.8773 - val_loss: 0.3497 - val_acc: 0.9105 - lr: 0.0010
Epoch 3/30
1297/1299 [============================>.] - ETA: 0s - loss: 0.3474 - acc: 0.9104
Epoch 3: val_acc improved from 0.91049 to 0.92647, saving model to best_model_epoch_03-0.29.h5
1299/1299 [==============================] - 27s 21ms/step - loss: 0.3474 - acc: 0.9104 - val_loss: 0.2936 - val_acc: 0.9265 - lr: 0.0010
Epoch 4/30
1299/1299 [===

In [40]:
# model.save('/content/gdrive/MyDrive/30-epoch-val-acc-9589-lr-1e-3-two-dropout-2e-1-batch-32-adam.h5')

### **Data Augmentation**


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:

train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
train_generator

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
   rescale=1.0 / 255,
   rotation_range=30,
   width_shift_range=0.2,
   height_shift_range=0.2,
   vertical_flip=True,
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 51952 images belonging to 20 classes.


In [ ]:
model = keras.Sequential([
            keras.Input(shape = (32, 32, 1)),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv1"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv2"),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu', name="Conv3"),
            layers.MaxPooling2D(name="Maxpool1"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv4"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv5"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv6"),
            layers.MaxPooling2D(name="Maxpool2"),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu', name="Conv7"),
            layers.Conv2D(128, 3, padding = 'same', activation = 'relu', name="Conv8"),
            layers.Flatten(name="Flatten1"),
            layers.Dense(128, activation = 'relu', name="Dense1"),
            layers.Dense(20, activation = 'softmax', name="Dense2"),
])

In [ ]:
adam =  keras.optimizers.Adam(lr = 1e-3)

In [ ]:
model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        optimizer = adam,
        metrics = ['acc'],
)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.5, min_lr = 1e-7)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
classifier = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
                    callbacks=[reduce_lr]
)

InvalidArgumentError: ignored

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

# Pytorch Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Resize([32,32]),
    #  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

path = "/content/gdrive/MyDrive/PreProcessDataset"
batch_size = 128

trainset = torchvision.datasets.ImageFolder(path, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 20)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x


net = Net()
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')